In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define image size and batch size
img_size = (256, 256)
batch_size = 32

# Data generator with automatic train-validation split
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)  # 20% of the data for validation

# Training generator (80% of data)
train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/original+augment",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True  # Shuffles the data for training
)

# Validation generator (20% of data)
val_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/original+augment",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False  # No need to shuffle for validation
)


Found 2019 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [ ]:
# Load the VGG16 model with pre-trained ImageNet weights, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(6, activation='softmax')(x)  # 7 classes for banana leaf diseases

# Define the final model
model = Model(inputs=base_model.input, outputs=output)

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# history = model.fit(
#     train_generator,
#     validation_data=val_generator,
#     epochs=6,
#     steps_per_epoch = train_generator.samples // batch_size,
#     validation_steps = val_generator.samples // batch_size
# )

In [ ]:
nb_epochs = 15

# Train the model
history = model.fit(
    train_generator,  # flow_from_directory generator for training data
    validation_data=val_generator,  # flow_from_directory generator for validation data
    epochs=nb_epochs,
    verbose=1
)
#no aug 4 ,aug 12

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


64/64 ━━━━━━━━━━━━━━━━━━━━ 863s 12s/step - accuracy: 0.3588 - loss: 1.9958 - val_accuracy: 0.7873 - val_loss: 0.6482
Epoch 2/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 14s 203ms/step - accuracy: 0.6605 - loss: 0.7879 - val_accuracy: 0.8867 - val_loss: 0.4325
Epoch 3/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 14s 207ms/step - accuracy: 0.7084 - loss: 0.6946 - val_accuracy: 0.9205 - val_loss: 0.3410
Epoch 4/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 21s 215ms/step - accuracy: 0.7520 - loss: 0.5588 - val_accuracy: 0.9364 - val_loss: 0.2476
Epoch 5/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 20s 210ms/step - accuracy: 0.7956 - loss: 0.4761 - val_accuracy: 0.9284 - val_loss: 0.2338
Epoch 6/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 21s 219ms/step - accuracy: 0.8176 - loss: 0.4348 - val_accuracy: 0.9324 - val_loss: 0.1871
Epoch 7/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 15s 219ms/step - accuracy: 0.8316 - loss: 0.3888 - val_accuracy: 0.9523 - val_loss: 0.1497
Epoch 8/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 17s 258ms/step - accuracy: 0.8629 - loss: 0.3323 - val_accuracy: 0.9523

In [ ]:
# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation accuracy: {val_accuracy:.2f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 180ms/step - accuracy: 0.9595 - loss: 0.1616
Validation accuracy: 0.97


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Predict the labels for the test set
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels from the test generator
y_true = val_generator.classes

# Generate and display a classification report
report = classification_report(y_true, y_pred_classes, target_names=val_generator.class_indices.keys())
print('Classification Report:\n', report)

16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 205ms/step
Classification Report:
                                precision    recall  f1-score   support

Banana Black Sigatoka Disease       0.96      0.92      0.94        83
                      healthy       1.00      1.00      1.00        79
                       insect       0.95      0.95      0.95        78
                         moko       1.00      0.96      0.98        84
                       panama       0.96      1.00      0.98        88
                       yellow       0.96      0.99      0.97        91

                     accuracy                           0.97       503
                    macro avg       0.97      0.97      0.97       503
                 weighted avg       0.97      0.97      0.97       503

